In [5]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as p
import time

import requests



### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [4]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

Failed to get events: 401, {"message":"Historical odds are only available on paid usage plans. See usage plans at https://the-odds-api.com","error_code":"HISTORICAL_UNAVAILABLE_ON_FREE_USAGE_PLAN","details_url":"https://the-odds-api.com/liveapi/guides/v4/api-error-codes.html#historical-unavailable-on-free-usage-plan"}



### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()

Scraping Odds API...
Organizing Data...


In [2]:
prizePicks = nba_props.dataframe
prizePicks
# df.to_csv('./PROPS_DATA/Playoffs_PP(5_13_25)')

,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,DraftKings,player_points,Jalen Williams,Over,21.5,-105
3,DraftKings,player_points,Jalen Williams,Under,21.5,-125
4,DraftKings,player_points,Pascal Siakam,Over,19.5,-110
...,...,...,...,...,...,...
1505,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-135
1506,DraftKings,player_blocks_steals,Aaron Nesmith,Over,1.5,114
1507,DraftKings,player_blocks_steals,Aaron Nesmith,Under,1.5,-145
1508,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,154


In [3]:
prizePicks.to_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')

In [7]:
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')
prizePicks.drop(columns=['Unnamed: 0'], inplace=True)
prizePicks

,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,DraftKings,player_points,Jalen Williams,Over,21.5,-105
3,DraftKings,player_points,Jalen Williams,Under,21.5,-125
4,DraftKings,player_points,Pascal Siakam,Over,19.5,-110
...,...,...,...,...,...,...
1505,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-135
1506,DraftKings,player_blocks_steals,Aaron Nesmith,Over,1.5,114
1507,DraftKings,player_blocks_steals,Aaron Nesmith,Under,1.5,-145
1508,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,154


### Grab player data using fetchplayerstats class

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2021-22', season_type='Playoffs', sleep_time=1, max_workers=4)
data

In [7]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_22.csv')

In [8]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')
data = data.drop(columns=['Unnamed: 0'])
data['OPP_TEAM_ID'] = data['OPP_TEAM_ID'].fillna(0).astype(int)
data.head()

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Gary Payton II,1627780,GSW vs. DEN,GSW,1610612744,DEN,1,42100161,2022-04-16,W,20,5,2,3,2,3,0.667,0,0,NaN,1,2,0.500,2,1,0,1,0,0,12,14.6,1.289,1.289,NaN,117.5,89.7,27.8,0.118,0.048,0.079,0.143,0.0,0.089,0.644,95.09,96.41,0.104,80.34,"APR 16, 2022",240,43,82,0.524,16,35,0.457,21,29,0.724,10,31,41,33,5,8,12,22,123,127.118644,8.0,3.0,35.0,0.462366,1610612743,127.118644,96.76,98.24,99.72
1,Zeke Nnaji,1630192,DEN @ GSW,DEN,1610612743,GSW,0,42100161,2022-04-16,L,4,3,0,0,1,1,1.000,1,1,1.000,0,0,NaN,0,0,0,0,0,1,4,3.0,3.000,3.000,NaN,144.4,90.0,54.4,0.000,0.000,0.000,0.000,0.0,0.111,1.500,114.69,120.53,0.103,100.44,"APR 16, 2022",240,43,93,0.462,11,35,0.314,10,13,0.769,9,26,35,26,8,3,10,22,107,107.300441,5.0,8.0,41.0,0.524390,1610612744,107.300441,99.72,98.24,96.76
2,Moses Moody,1630541,GSW vs. DEN,GSW,1610612744,DEN,1,42100161,2022-04-16,W,4,1,1,0,0,1,0.000,0,1,0.000,1,2,0.500,0,0,1,0,0,0,-6,5.5,0.532,0.532,NaN,77.8,144.4,-66.7,0.000,0.000,0.000,0.500,0.0,0.222,0.266,114.14,119.45,0.054,99.54,"APR 16, 2022",240,43,82,0.524,16,35,0.457,21,29,0.724,10,31,41,33,5,8,12,22,123,127.118644,8.0,3.0,35.0,0.462366,1610612743,127.118644,96.76,98.24,99.72
3,Bryn Forbes,1627854,DEN @ GSW,DEN,1610612743,GSW,0,42100161,2022-04-16,L,14,5,0,0,2,4,0.500,1,2,0.500,0,0,NaN,0,0,1,0,0,1,0,8.0,1.250,1.250,NaN,127.6,123.3,4.3,0.000,0.000,0.000,0.000,0.0,0.129,0.625,103.22,102.73,0.042,85.61,"APR 16, 2022",240,43,93,0.462,11,35,0.314,10,13,0.769,9,26,35,26,8,3,10,22,107,107.300441,5.0,8.0,41.0,0.524390,1610612744,107.300441,99.72,98.24,96.76
4,James Harden,201935,PHI vs. TOR,PHI,1610612755,TOR,1,42100131,2022-04-16,W,40,22,14,5,6,17,0.353,4,7,0.571,6,7,0.857,2,3,1,1,1,0,14,54.0,1.096,1.096,G,144.6,124.0,20.6,0.051,0.091,0.069,0.452,14.0,0.241,0.548,90.66,89.10,0.144,74.25,"APR 16, 2022",240,43,84,0.512,16,32,0.500,29,34,0.853,10,29,39,29,6,4,3,18,131,142.453241,1.0,8.0,35.0,0.487805,1610612761,142.453241,91.96,92.54,93.12


In [ ]:
from NBAData.features import add_PTS_features
from NBAData.TBD import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = add_PTS_features(data)
data = assign_position(data)
data

In [ ]:
#dummy variables
team_dummies = pd.get_dummies(data['TEAM_ABBREVIATION'], prefix='TEAM').astype(int)
opp_dummies = pd.get_dummies(data['OPP_ABBREVIATION'], prefix='OPP').astype(int)
data = pd.concat([data, team_dummies, opp_dummies], axis=1)

data = data.drop(['TEAM_ABBREVIATION', 'OPP_ABBREVIATION'], axis=1)

data


In [11]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_22_FEATURES.csv')

### If you already have the data just run the cell below

In [100]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')
Props = prizePicks[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[prizePicks['CATEGORY'] == 'player_points']

### For Single Bets

In [103]:
import numpy as np
import pandas as pd
import scipy.stats as stats

results = []

for idx, row in Props.iterrows():
    name = row['NAME']
    bookmaker = row['BOOKMAKER']
    category = row['CATEGORY']
    line = row['LINE']
    over_under = row['OVER/UNDER']
    odds = row['ODDS']

    # Filter player data
    df = data.loc[data['PLAYER_NAME'] == name]
    if df.empty:
        continue

    # Mean and std dev of actual results
    mean = round(df['PTS'].mean(), 2)
    std = round(df['PTS'].std(), 2)
    if std == 0:
        continue

    # Z-score-based probability
    z_score = (line - mean) / std
    norm_prob = 1 - stats.norm.cdf(z_score)
    if over_under == "Under":
        norm_prob = 1 - norm_prob

    # Monte Carlo simulation
    sim_points = np.random.normal(mean, std, 10000)
    sim_prob = np.mean(sim_points > line)
    if over_under == "Under":
        sim_prob = 1 - sim_prob

    # Weighted average of sim and norm prob
    final_prob = 0.8 * sim_prob + 0.2 * norm_prob
    
    # EV calculation
    stake = 100
    profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
    payout = stake + profit
    ev = (final_prob * profit) - ((1 - final_prob) * stake)

    # Kelly Criterion
    def kelly_criterion(prob, payout, stake):
        net_profit = payout - stake
        loss_prob = 1 - prob
        kelly = (net_profit * prob - loss_prob * stake) / net_profit
        return max(0, round(kelly, 4))

    kelly = kelly_criterion(final_prob, payout, stake)

    results.append({
        'NAME': name,
        'BOOKMAKER': bookmaker,
        'CATEGORY': category,
        'LINE': line,
        'OVER/UNDER': over_under,
        'ODDS': odds,
        'SIM PROB': round(final_prob, 3),
        'EV': round(ev, 2),
        'KELLY CRITERION': kelly,
    })

current = pd.DataFrame(results).sort_values(by='EV', ascending=False).reset_index(drop=True)
current
# current.to_csv('PROPS_EV/singleBets(6_9_25).csv')

,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,SIM PROB,EV,KELLY CRITERION
0,Shai Gilgeous-Alexander,Bovada,player_points,29.5,Under,210,0.467,44.80,0.2134
1,Isaiah Hartenstein,BetRivers,player_points,7.5,Over,112,0.683,44.75,0.3995
2,Shai Gilgeous-Alexander,Bovada,player_points,31.5,Under,150,0.569,42.21,0.2814
3,Isaiah Hartenstein,BetRivers,player_points,6.5,Over,-121,0.778,42.11,0.5095
4,Shai Gilgeous-Alexander,Bovada,player_points,30.5,Under,175,0.506,39.18,0.2239
...,...,...,...,...,...,...,...,...,...
373,Isaiah Hartenstein,FanDuel,player_points,6.5,Under,102,0.221,-55.38,0.0000
374,Isaiah Hartenstein,BetRivers,player_points,6.5,Under,-109,0.225,-56.93,0.0000
375,Shai Gilgeous-Alexander,Bovada,player_points,38.5,Over,170,0.152,-58.91,0.0000
376,Isaiah Hartenstein,Bovada,player_points,5.5,Under,170,0.151,-59.28,0.0000


### We try to find the pairs with the highest EVs, since prizepicks only accepts 2+ legs

In [105]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_PP_DFS(6_9_25).csv')
pointsProps = prizePicks[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[prizePicks['CATEGORY'] == 'player_points']

In [106]:
from scipy.stats import norm
# This is the monte carlo simulation for the pairs, then multiplies with the z-score probability, followed by the kelly criterion

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def monte_carlo_sim(name1, name2, simulations=10000, stake=100, payout=300, weight=0.8):
    df1 = data.loc[data['PLAYER_NAME'] == name1]
    df2 = data.loc[data['PLAYER_NAME'] == name2]

    line1 = pointsProps.loc[pointsProps['NAME'] == name1, 'LINE'].values[0]
    line2 = pointsProps.loc[pointsProps['NAME'] == name2, 'LINE'].values[0]

    mean1, std1 = df1['PTS'].mean(), df1['PTS'].std()
    mean2, std2 = df2['PTS'].mean(), df2['PTS'].std()
    
    if std1 == 0 or std2 == 0:
        return None

    # Monte Carlo simulation
    sim1 = np.random.normal(mean1, std1, simulations)
    sim2 = np.random.normal(mean2, std2, simulations)

    over1, under1 = sim1 > line1, sim1 <= line1
    over2, under2 = sim2 > line2, sim2 <= line2

    mc_probs = {
        'OVER/OVER': np.mean(over1 & over2),
        'UNDER/UNDER': np.mean(under1 & under2),
        'OVER/UNDER': np.mean(over1 & under2),
        'UNDER/OVER': np.mean(under1 & over2),
    }

    # Z-score-based probabilities
    z_over1 = zscore_prob(mean1, std1, line1, side='over')
    z_under1 = 1 - z_over1
    z_over2 = zscore_prob(mean2, std2, line2, side='over')
    z_under2 = 1 - z_over2

    z_probs = {
        'OVER/OVER': z_over1 * z_over2,
        'UNDER/UNDER': z_under1 * z_under2,
        'OVER/UNDER': z_over1 * z_under2,
        'UNDER/OVER': z_under1 * z_over2,
    }

    # Blended probabilities
    probs = {
        k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
        for k in mc_probs
    }

    evs = {k: round((probs[k] * payout) - stake, 2) for k in probs}

    return {
        'PLAYER 1': name1,
        'PLAYER 2': name2,
        'OVER/OVER': {'prob': round(probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
        'UNDER/UNDER': {'prob': round(probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
        'OVER/UNDER': {'prob': round(probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
        'UNDER/OVER': {'prob': round(probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
    }

    
def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))
    

In [109]:
# pair EV
# ((player1.prob * player2.prob) * payout) - ((1 - player1.prob * player2.prob) * stake)
pairEV = []
seen = set()

for name1 in current['NAME'].unique():
    for name2 in current['NAME'].unique():
        if name1 == name2:
            continue
        
        name1_sorted, name2_sorted = sorted([name1, name2])
        pair_key = (name1_sorted, name2_sorted)
        if pair_key in seen:
            continue
        seen.add(pair_key)

        
        sim_result = monte_carlo_sim(name1, name2, weight=0.8)
        if sim_result is None:
            continue
        evs = {
            'OVER': sim_result['OVER/OVER']['ev'],
            'UNDER': sim_result['UNDER/UNDER']['ev'],
            'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
            'UNDER/OVER': sim_result['UNDER/OVER']['ev']
        }
        
        probs = {
            'OVER': sim_result['OVER/OVER']['prob'],
            'UNDER': sim_result['UNDER/UNDER']['prob'],
            'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
            'UNDER/OVER': sim_result['UNDER/OVER']['prob']
        }
        
        bestType = max(evs, key=evs.get)
        bestEV = evs[bestType]
        bestProb = probs[bestType]
        
        pairEV.append({
            'PLAYER 1': name1_sorted,
            'PLAYER 1 LINE': pointsProps.loc[pointsProps['NAME'] == name1_sorted, 'LINE'].values[0],
            'PLAYER 2': name2_sorted,
            'PLAYER 2 LINE': pointsProps.loc[pointsProps['NAME'] == name2_sorted, 'LINE'].values[0],
            'TYPE': bestType,
            'EV': round(bestEV, 2),
            'PROBABILITY': round(bestProb, 4), # Probability of both players scoring over their lines
            'KELLY CRITERION': kelly_criterion(bestProb, 300, 100)
        })
todaysPairEV = pd.DataFrame(pairEV).sort_values(by='EV', ascending=False).reset_index(drop=True)
todaysPairEV
# todaysPairEV.to_csv('PROPS_EV/todaysPairEV(6_9_25).csv')

        

,PLAYER 1,PLAYER 1 LINE,PLAYER 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Isaiah Hartenstein,7.0,Shai Gilgeous-Alexander,34.5,UNDER/OVER,55.52,0.5184,0.5160
1,Isaiah Hartenstein,7.0,Isaiah Joe,3.5,OVER,41.30,0.4710,0.4684
2,Isaiah Joe,3.5,Shai Gilgeous-Alexander,34.5,UNDER/OVER,36.41,0.4547,0.4520
3,Aaron Nesmith,12.0,Isaiah Hartenstein,7.0,OVER,33.44,0.4448,0.4420
4,Isaiah Hartenstein,7.0,T.J. McConnell,7.0,OVER,33.35,0.4445,0.4417
...,...,...,...,...,...,...,...,...
166,Aaron Wiggins,6.5,Alex Caruso,9.0,UNDER,-16.24,0.2792,0.2756
167,Alex Caruso,9.0,Chet Holmgren,15.0,OVER/UNDER,-16.29,0.2790,0.2754
168,Luguentz Dort,8.5,Obi Toppin,9.5,UNDER,-16.64,0.2779,0.2743
169,Alex Caruso,9.0,Luguentz Dort,8.5,UNDER,-18.62,0.2713,0.2677
